## EfficientNetB3 Model 

In [15]:
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow.compat.v1 as tf
import argparse
import math
import numpy as np
import os
from tensorflow.keras import optimizers
from tensorflow.keras import layers, initializers
from tensorflow.keras import models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from keras.models import model_from_json
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import Resizing
from PIL import Image

In [27]:
tf.enable_eager_execution()

In [28]:
# NUM_TRAIN = 16000
# NUM_VAL = 3200
IMG_DIM = 65
NUM_CLASSES = 4
# TOTAL_TRAIN = 100
# TOTAL_VAL = 50
TOTAL_TRAIN = 86317
TOTAL_VAL = 10778

In [29]:
def file_list_from_folder(folder, data_path):
    folderpath = os.path.join(data_path, folder)
    filelist = []
    for filename in os.listdir(folderpath):
        if filename.startswith('part-') and not filename.endswith('gstmp'):
            filelist.append(os.path.join(folderpath, filename))
    return filelist

def load_data(data_path):
    train = file_list_from_folder("train", data_path)
    val = file_list_from_folder("val", data_path)
    return train, val

In [30]:
features = {
  'B1': tf.io.FixedLenFeature([], tf.string),
  'B2': tf.io.FixedLenFeature([], tf.string),
  'B3': tf.io.FixedLenFeature([], tf.string),
  'B4': tf.io.FixedLenFeature([], tf.string),
  'B5': tf.io.FixedLenFeature([], tf.string),
  'B6': tf.io.FixedLenFeature([], tf.string),
  'B7': tf.io.FixedLenFeature([], tf.string),
  'B8': tf.io.FixedLenFeature([], tf.string),
  'B9': tf.io.FixedLenFeature([], tf.string),
  'B10': tf.io.FixedLenFeature([], tf.string),
  'B11': tf.io.FixedLenFeature([], tf.string),
  'label': tf.io.FixedLenFeature([], tf.int64),
}

In [31]:
def parse_tfrecords(filelist, batch_size, buffer_size, include_viz=False):
    # try a subset of possible bands
    def _parse_(serialized_example, keylist=['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8']):
        example = tf.io.parse_single_example(serialized_example, features)

        def getband(example_key):
            img = tf.io.decode_raw(example_key, tf.uint8)
            return tf.reshape(img[:IMG_DIM**2], shape=(IMG_DIM, IMG_DIM, 1))

        bandlist = [getband(example[key]) for key in keylist]
        # combine bands into tensor
        image = tf.concat(bandlist, -1)

        # one-hot encode ground truth labels
        label = tf.cast(example['label'], tf.int32)
        label = tf.one_hot(label, NUM_CLASSES)

        return {'image': image}, label

    tfrecord_dataset = tf.data.TFRecordDataset(filelist)
    tfrecord_dataset = tfrecord_dataset.map(lambda x:_parse_(x)).shuffle(buffer_size).repeat(-1).batch(batch_size)
    tfrecord_iterator = tfrecord_dataset.make_one_shot_iterator()
    image, label = tfrecord_iterator.get_next()
    return image, label

In [32]:
def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()

In [33]:
def input_preprocess(image, label):
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label

In [34]:
##CODE

In [35]:
print("loading data")
train_tfrecords, val_tfrecords = load_data("../droughtwatch/data/")
train_images, train_labels = parse_tfrecords(train_tfrecords, TOTAL_TRAIN, TOTAL_TRAIN)
val_images, val_labels = parse_tfrecords(val_tfrecords, TOTAL_VAL, TOTAL_VAL)
print("Finished loading data!")
print("Preparing hold out...")
# #Divide the data in a train set, a validation set, and a test set and store it in variables as tensors
k = int((2/3)*TOTAL_TRAIN)
X_tr = train_images["image"][:k]
y_tr = train_labels[:k]
X_val = train_images["image"][k:]
y_val = train_labels[k:]
X_test = val_images["image"]
y_test = val_labels

print("deleting unused variables...")
del train_tfrecords
del val_tfrecords
del val_images
del val_labels


loading data
Finished loading data!
Preparing hold out...
deleting unused variables...


In [36]:
# #Keep only images that are not all blank nor all black and convert the tensors as np arrays
print("Filtering blank and black images")
indices = np.where([i[i.std() >= 10].all() for i in X_tr.numpy()])
X_tr, y_tr = X_tr.numpy(), y_tr.numpy()
X_tr, y_tr = X_tr[indices], y_tr[indices]

indices = np.where([i[i.std() >= 10].all() for i in X_val.numpy()])
X_val, y_val = X_val.numpy(), y_val.numpy()
X_val, y_val = X_val[indices], y_val[indices]

indices = np.where([i[i.std() >= 10].all() for i in X_test.numpy()])
X_test, y_test = X_test.numpy(), y_test.numpy()
X_test, y_test = X_test[indices], y_test[indices]
print("finished filtering blank and black images")
# #Keep only rgb channels for the vgg16 model
print("converting images to rgb ...")
X_trrgb = X_tr[:,:,:,1:4]
X_valrgb = X_val[:,:,:,1:4]
X_testrgb = X_test[:,:,:,1:4]
print("finished converting images to rgb")

print("deleting unused variables")
del X_tr
del X_val
del X_test

Filtering blank and black images
finished filtering blank and black images
converting images to rgb ...
finished converting images to rgb
deleting unused variables


In [ ]:
#Build EfficientnetB3 model
IMG_SIZE = 300
datagen = tf.keras.preprocessing.image.ImageDataGenerator()
datagen2 = tf.keras.preprocessing.image.ImageDataGenerator()
# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(X_trrgb)
datagen2.fit(X_valrgb)

# ----------------
inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = inputs
x = Resizing(300, 300)(x)
activationnetB3 = EfficientNetB3(include_top=False, weights = "imagenet")(x)
outputsflatten = layers.Flatten()(activationnetB3)
outputsdense1 = layers.Dense(64, activation = "relu")(outputsflatten)
outputsdense2 = layers.Dense(4, activation = "softmax")(outputsdense1)
model = tf.keras.Model(inputs, outputsdense2)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=20, verbose=1, restore_best_weights=True)


history = model.fit(datagen.flow(X_trrgb, y_tr, batch_size=32),\
         epochs=1000, validation_data = datagen2.flow(X_valrgb, y_val, batch_size = 32), verbose = 1,\
                    callbacks=[es])


del X_trrgb
del X_valrgb

## BElOW COMMENTED CODE --> USING IMAGE FOLDER

In [ ]:
# from PIL import Image
# img = Image.fromarray(X_trrgb[0], 'RGB')
# img.save('../image/my.png')
# img.show()

In [ ]:
# def saving_images(images,labels):
    
#     for images, labels in range(TOTAL_TRAIN + TOTAL_VAL):
#         img = Image.fromarray(train_images, 'RGB')
#         img.save(f'../image/{labels}/{images}.png')


In [46]:
#Build EfficientnetB3 model
# IMG_SIZE = 300
# datagen = tf.keras.preprocessing.image.ImageDataGenerator()
# datagen2 = tf.keras.preprocessing.image.ImageDataGenerator()
# # compute quantities required for featurewise normalization
# # (std, mean, and principal components if ZCA whitening is applied)
# datagen.fit(X_trrgb)
# datagen2.fit(X_valrgb)

# # ----------------
# inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
# x = inputs
# x = Resizing(300, 300)(x)
# activationnetB3 = EfficientNetB3(include_top=False, weights = "imagenet")(x)
# outputsflatten = layers.Flatten()(activationnetB3)
# outputsdense1 = layers.Dense(64, activation = "relu")(outputsflatten)
# outputsdense2 = layers.Dense(4, activation = "softmax")(outputsdense1)
# model = tf.keras.Model(inputs, outputsdense2)
# model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# es = EarlyStopping(monitor='val_accuracy', mode='max', patience=20, verbose=1, restore_best_weights=True)


# history = model.fit(datagen.flow_from_directory('../image/',target_size = (300,300), color_mode='rgb',   batch_size=32),\
#          epochs=1000, verbose = 1,\
#                     callbacks=[es])

# del X_trrgb
# del X_valrgb


Found 67 images belonging to 4 classes.
Epoch 1/1000
3/3 [==============================] - 30s 10s/step - loss: 8.7366 - accuracy: 0.5672
Epoch 2/1000
3/3 [==============================] - 39s 13s/step - loss: 29.6760 - accuracy: 0.4478
Epoch 3/1000
3/3 [==============================] - 64s 21s/step - loss: 1.3700 - accuracy: 0.8358
Epoch 4/1000
2/3 [===================>..........] - ETA: 16s - loss: 0.2889 - accuracy: 0.9143

KeyboardInterrupt: 